# Model Iteration 1

In [1]:
import pandas
import re

def stem_phrases(data):
    df = data
    
    # looks for any commas, periods, and apostrophes with any leading or following spaces
    remove_punct = r'(( )*((\.)|(\'s*)|(,))( )*)'

    # remove leading and trailing spaces
    remove_spaces = r'(^( )*)|(( )*$)'

    # replaces the pattern mentioned above with a single space
    df['Phrase'] = df['Phrase'].str.replace(remove_punct, ' ')
    df = df[df['Phrase'] != ' ']
    #print df[df.loc(df['Phrase'] != ' ')]
    #df = df.loc(df['Phrase'] != ' ')
    df['Phrase'] = df['Phrase'].str.replace(remove_spaces, '')

    # Removes empty string phrases from the list
    B = []
    before = ""
    for r in df['Phrase']:
        if r != before:
            B.append(True)
            before = r
        else:
            B.append(False)
    df = df[B]
    return df

In [2]:
df = pandas.DataFrame.from_csv('train.tsv', sep='\t')
#stemmed = stem_phrases(df)
#stemmed.head(5)

In [3]:
import numpy as np

def get_words(df):
    data = df[~(df['Phrase'].str.contains(' '))]
    return data

def get_sentences(df):
    sentences = df.drop_duplicates(['SentenceId'])
    return sentences

In [4]:
words = get_words(df)
words.head(45)

,SentenceId,Phrase,Sentiment
PhraseId,,,
4,1,A,2
5,1,series,2
7,1,of,2
9,1,escapades,2
12,1,demonstrating,2
14,1,the,2
15,1,adage,2
17,1,that,2
19,1,what,2


In [5]:
import nltk
from nltk.tag.perceptron import PerceptronTagger

# Load the tagger for faster tagging
tagger = PerceptronTagger()

In [16]:
def tag_sentences(df):
    data = df
    data["POS"] = data["Phrase"].apply(lambda x: nltk.tag._pos_tag(nltk.word_tokenize(x), None, tagger))
    return data

# I wasn't able to put the tags on each word after tagging each sentence
# This is due to the stemming done earlier
def tag_words(words, sentences):
    possible_tags = ['CC', 'CD', 'DT', 'EX',
                     'FW', 'IN', 'JJ', 'JJR',
                     'JJS', 'LS', 'MD', 'NN',
                     'NNP', 'NNPS', 'NNS', 'PDT',
                     'POS', 'PRP', 'PRP$', 'RB',
                     'RBR', 'RBS', 'RP', 'SYM',
                     'TO', 'UH', 'VB', 'VBD',
                     'VBG', 'VBN', 'VBP', 'VBZ',
                     'WDT', 'WDT', 'WP', 'WP$',
                     'WRB']
    data = []
    words_with_pos = words
    j = 0
    for sentence_id, sentence_pos in zip(sentences["SentenceId"], sentences["POS"]):
        sentence_words = words[words["SentenceId"] == sentence_id]
        #data.append(sentence_pos)
        for sentence_word_pos in sentence_pos:
            if j == 0:
                print sentence_word_pos
                j += 1
            data.append(sentence_word_pos[1])
    for m,n in zip(words_with_pos.Phrase, data):
        if j < 45:
            print m, n
            j += 1
    print len(words_with_pos), len(data)
    words_with_pos["POS"] = data
    return words_with_pos

In [ ]:
def tag_words2(words):
    

In [7]:
sentences = get_sentences(df)
print sentences[:3]

          SentenceId                                             Phrase  \
PhraseId                                                                  
1                  1  A series of escapades demonstrating the adage ...   
64                 2  This quiet , introspective and entertaining in...   
82                 3  Even fans of Ismail Merchant 's work , I suspe...   

          Sentiment  
PhraseId             
1                 1  
64                4  
82                1  


In [8]:
tagged_sentences = tag_sentences(sentences)

-c:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [9]:
tagged_sentences.head(10)

,SentenceId,Phrase,Sentiment,POS
PhraseId,,,,
1,1,A series of escapades demonstrating the adage ...,1,"[(A, DT), (series, NN), (of, IN), (escapades, ..."
64,2,"This quiet , introspective and entertaining in...",4,"[(This, DT), (quiet, JJ), (,, ,), (introspecti..."
82,3,"Even fans of Ismail Merchant 's work , I suspe...",1,"[(Even, RB), (fans, NNS), (of, IN), (Ismail, N..."
117,4,A positively thrilling combination of ethnogra...,3,"[(A, DT), (positively, RB), (thrilling, VBG), ..."
157,5,Aggressive self-glorification and a manipulati...,1,"[(Aggressive, JJ), (self-glorification, NN), (..."
167,6,A comedy-drama of nearly epic proportions root...,4,"[(A, DT), (comedy-drama, NN), (of, IN), (nearl..."
199,7,"Narratively , Trouble Every Day is a plodding ...",1,"[(Narratively, RB), (,, ,), (Trouble, JJ), (Ev..."
214,8,"The Importance of Being Earnest , so thick wit...",3,"[(The, DT), (Importance, NN), (of, IN), (Being..."
248,9,But it does n't leave you with much .,1,"[(But, CC), (it, PRP), (does, VBZ), (n't, RB),..."


In [17]:
tagged_words = tag_words(words, tagged_sentences)
#for i in range(len(tagged_words[0])+1):
#    print tagged_words[i]
tagged_words.head(5)

#Why are the lengths off by so much?

('A', 'DT')
A DT
series NN
of IN
escapades NNS
demonstrating VBG
the DT
adage NN
that IN
what WP
is VBZ
good JJ
for IN
goose DT
also NN
gander VBZ
, RB
some JJ
which IN
occasionally DT
amuses NN
but ,
none DT
amounts IN
to WDT
much RB
story VBZ
. CC
This NN
quiet IN
introspective WDT
and NNS
entertaining TO
independent JJ
worth IN
seeking DT
Even NN
fans .
Ismail DT
Merchant JJ
's ,
work JJ
I CC
suspect JJ
would JJ
16531 162121


ValueError: Length of values does not match length of index

In [ ]:
def tag_sentences(df):
    data = df
    data["POS"] = data["Phrase"].apply(lambda x: nltk.pos_tag(nltk.word_tokenize(x)))

Time to handle negations!

In [ ]:
def is_negative(df):
    for phr in df: